In [3]:
pip install tensorflow

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
     ---------------------------------------- 46.2/46.2 kB ? eta 0:00:00
   ---------------------------------------- 0.0/377.1 MB ? eta -:--:--
   ---------------------------------------- 1.0/377.1 MB ? eta -:--:--
   ---------------------------------------- 1.0/377.1 MB ? eta -:--:--
   ---------------------------------------- 1.0/377.1 MB ? eta -:--:--
   ---------------------------------------- 1.0/377.1 MB ? eta -:--:--
   ---------------------------------------- 1.0/377.1 MB ? eta -:--:--
   ---------------------------------------- 3.1/377.1 MB 12.5 MB/s eta 0:00:30
   ---------------------------------------- 3.1/377.1 MB 12.5 MB/s eta 0:00:30
   ---------------------------------------- 3.1/377.1 MB 12.5 MB/s eta 0:00:30
   ---------------------------------------- 3.1/377.1 MB 12.5 MB/s eta 0:00:30
   --------

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the data
data = pd.read_csv('Final_Merged_Data.csv')

# Handle missing values
data = data.dropna()

# Convert 'timestamp_sentinel2' to datetime
data['timestamp_sentinel2'] = pd.to_datetime(data['timestamp_sentinel2'])

# Define the target variable (y)
target = 'Turbidity_Buoy_(NTU)'

# # Select features
# features = [
#     *[f'point{point}_B{band}' for band in [2, 3, 4, 8, 8, 8, 8, 11, 12] for point in range(1, 23)],
#     *[f'point{point}_cs' for point in range(1, 23)],
#     *[f'point{point}_cs_cdf' for point in range(1, 23)],
#     'B2_AVG', 'B3_AVG', 'B4_AVG', 'B8_AVG', 'B8A_AVG', 'B11_AVG', 'B12_AVG',
#     'cs_AVG', 'cs_cdf_AVG',
#     'Ground_Measurements_time_diff_(seconds)', 'Lake_Height_(m)', 
#     'PercentFull_Active_Lake_Storage_(%)', 'Snow_Volume_Opuha_Catchment_(mm)',
#     'WDir(Deg)', 'WSpd(m/s)', 'GustDir(Deg)', 'GustSpd(m/s)', 'WindRun(Km)', 
#     'Rain(mm)', 'Tdry(C)', 'TWet(C)', 'RH(%)', 'Tmax(C)', 'Tmin(C)', 
#     'Pmsl(hPa)', 'Pstn(hPa)', 'Water_Temp_Buoy_(degC)', 'Water_Temp_Platform_(degC)'
# ]

features = [
    'B2_AVG', 'B3_AVG', 'B4_AVG', 'B8_AVG', 'B8A_AVG', 'B11_AVG', 'B12_AVG',
    'cs_AVG', 'cs_cdf_AVG',
    'Ground_Measurements_time_diff_(seconds)', 'Lake_Height_(m)', 
    'PercentFull_Active_Lake_Storage_(%)', 'Snow_Volume_Opuha_Catchment_(mm)',
    'WDir(Deg)', 'WSpd(m/s)', 'GustDir(Deg)', 'GustSpd(m/s)', 'WindRun(Km)', 
    'Rain(mm)', 'Tdry(C)', 'TWet(C)', 'RH(%)', 'Tmax(C)', 'Tmin(C)', 
    'Pmsl(hPa)', 'Pstn(hPa)', 'Water_Temp_Buoy_(degC)', 'Water_Temp_Platform_(degC)'
]



X = data[features]
y = data[target]

# Normalise the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=features)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Make predictions
y_pred_nn = model.predict(X_test)

# Evaluate the model
mse_nn = mean_squared_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)

print(f"Neural Network Mean Squared Error: {mse_nn}")
print(f"Neural Network R-squared: {r2_nn}")


c:\Program Files\Anaconda3\envs\ee\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step - loss: 1292.2726 - val_loss: 2155.6313
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 626.9249 - val_loss: 2113.6353
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 734.2546 - val_loss: 2055.9421
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 480.6805 - val_loss: 1994.6781
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 641.3809 - val_loss: 1942.1985
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1238.3042 - val_loss: 1904.7794
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 684.5928 - val_loss: 1892.6409
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 1231.9963 - val_loss: 1881.6650
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 540.2810 - val_loss: 1874.2856
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 522.6221 - val_loss: 1860.2294
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 482.5023 - val_loss: 1831